# Step 1 - Scraping

Realizar scraping do google scholar e coletar informações de um autor.

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [2]:
url    = 'https://scholar.google.com.br'
autor   = 'Jorge Soares'
# autor   = 'Jeffrey Heer'

In [3]:
# Acessar via browser o site do Google Scholar
options = Options()
# options.headless = True
driver = webdriver.Firefox(options=options)
driver.get(url)

# Inserir nome do autor a ser buscado e clicar na busca
inputElement = driver.find_element_by_id("gs_hdr_tsi")
inputElement.send_keys(autor)
driver.find_element_by_id('gs_hdr_tsb').click()

# Buscar a url com o endereço para a página do autor pesquisado
soup = BeautifulSoup(driver.page_source)
try:
    link = soup.find('h4',class_='gs_rt2').contents[0].get('href')
    autor_url = url+link
except:
    print('Autor não encontrado')
    driver.quit()

In [13]:
# Acessar a página do autor
driver.get(autor_url)

# Clicar no botão de "More" para exibir a página completa
btn_More   = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "gsc_bpf_more")))
for i in range(100):
    time.sleep(0.3)
    if btn_More.get_attribute('disabled')=='true':
        break
    else:
        driver.find_element_by_id('gsc_bpf_more').click()
        
# Coletar html da página
soup = BeautifulSoup(driver.page_source)

# Criar lista com papers contendo artigos e autores
table = soup.find('table',attrs={"id":'gsc_a_t'})
papers = []
for article in table.find_all('td',attrs={"class":'gsc_a_t'}):
    try:
        autores = article.find('div',attrs={'class':'gs_gray'}).contents[0].split(',')

        # retira o espaço branco no início e final dos nomes, e remove o '...'
        autores = [autor.strip() for autor in autores]
        if autores[-1]=='...': autores.pop()

        title   = article.find('a').contents[0]
        papers.append({'title':title,'authors':autores})
    except:
        pass

    
# Fechar browser
driver.quit()

In [14]:
papers

[{'title': 'Pavimentação asfáltica: formação básica para engenheiros',
  'authors': ['LLB Bernucci', 'LMG Motta', 'JAP Ceratti', 'JB Soares']},
 {'title': 'Characterization and thermal behavior of polymer-modified asphalt',
  'authors': ['MCC Lucena', 'SA Soares', 'JB Soares']},
 {'title': 'Crack modeling of asphaltic mixtures considering heterogeneity of the material',
  'authors': ['JB Soares', 'FA Freitas', 'DH Allen']},
 {'title': 'Evaluation of different digital image processing software for aggregates and hot mix asphalt characterizations',
  'authors': ['IS Bessa', 'VTFC Branco', 'JB Soares']},
 {'title': 'Model for predicting damage evolution in heterogeneous viscoelastic asphaltic mixtures',
  'authors': ['FV de Souza', 'JB Soares', 'DH Allen', 'F Evangelista Jr']},
 {'title': 'Aging of asphaltic binders investigated with atomic force microscopy',
  'authors': ['LM Rebelo',
   'JS De Sousa',
   'AS Abreu',
   'M Baroni',
   'AEV Alencar',
   'SA Soares']},
 {'title': 'Asphalt 